Hello and welcome to this tutorial on how to use the files in this repository to create PACEMAKER potentials!

At first we need to put our data into a format that PACEMAKER can read. Concetely a pickeld and gzipped pandas dataframe that contains the entire data. Data being the energy, the forces, the atom positions and the corrected energy.

To convert .traj files from ASE to PACEMAKER .pckl.gzip files excecute the process_data.sh file. Make sure to change the folder variable in this file to the folder that contains your data. The data can also be stored in subfolders, this does not matter since the script will traverse through these. The script will then excecute the ASE_to_Pacemaker_script.py (thanks to Michael J. Waters for this) that converts the files to a pandas dataframe.

Now the since the dataframes are in their original folders still, I'd recommend using the group_data.py script. This script copies all the .pckl.gzip dataframes we just created into a single folder.

In [ ]:
from group_data import copy_pickl_gzip_files
# Define the source directory to search for files
source_dir = "/home/flo/pacemaker/Data/" #Change

# Define the destination directory to copy the files
destination_dir = '/home/flo/pacemaker/data_grouped' #Change

#Execute
copy_pickl_gzip_files(source_dir, destination_dir)


Now we have everything setup. The next step is to sample the data and reduce the redundancy of our MD data. I'd consider this step to be rather optional, however given our data set we do not gain too much by leaving all the data in and training on it. As mentioned redundancy is high and the only effect it will have is that tha model will take longer to train for virtually the same performance.

There are many scripts that can be called to reduce the redundancy. And they are all in the reduce_data folder. My personal recommendations are the redaac_every_xth.py and the sample_more_peaks.py scripts. In general the scripts seem to sample many datapoints in the 2nd half of the .traj files. I guess it is because there the density of data points is much higher because the peaks are much less pronounced.

For "beginners" sample_more_peaks.py might be the better option. The parameters for the function are reduce_redundancy_min_max(df, metric, reduction, norm=None, full_return=False). First one is the dataframe, 2nd the metric which can be "forces" or "ase_atoms" for distances. With the reduction parameter you can choose the percentage of how mch data there should be left i.e. 0.5 for a reduction to 50% of the original data set. The norm can be None, 'fro' or np.inf. Ignore the full_return it doesn't do anything in this version.
This method detects the peaks and then assigns an importance to the data points depending on the distance to the peaks. As such this method is likely better suited for sampling peaks in the data.

The redaac_every_xth.py gives you the function reduce_redundancy(df, metric, threshold, norm=None, every_xth=5). The first argument is the dataframe that you want to reduce. The second one ist the metric you want to choose for reducing. These can be "energy", "forces", "distance", "energy_corrected". The third parameter is a threshold. Then you can choose a norm (everything compatible with numpy and the data such as None, 'fro', np.inf). The last argument every_xth allows for the option to take in every x_th set value in case for the last x_th data points none was taken.

This method works by calculating the difference between two neighbouring points in the data according to the norm parameter. If the difference is above the threshold the value gets taken, otherwise it moves on to the next point. Occasionally the every_xth setting interferes here.
You can also use this by either setting the threshold really high such that you essentially only sample using the every x_th setting and only sample let's say every 5th value from the data set.
If you do not want this option I'd recommend to set the every_xth parameter super high or use reduce_data_accurate.py which is essentially the same without the every_xth option.

Using this method you will have to play around with the threshold parameter and maybe the norms and metrics

In general it is totally an option to combine both methods to reduce the data! Such as using an 50% reduction with sample_more_peaks.py and then narrow it down using  redaac_every_xth.py or the other way around. Just make sure to switch the folder/file variable in such a way to include the already reduced data.

Other reduction methods include reduce_data.py which in every step of the reduction only looks at a pair of neighbouring points to calculate the difference, whereas the redaac_every_xth.py and reduce_data_accurate.py always compare the new incoming point to the last one that was taken into the reduced data set.
Also there is reduce_data_accurate_gradient.py which divides the difference between two points by the amount of (time/simulation-) steps between them. This one I do not really recommend, as finding parameters that give desired results is quite fidgety.

As another recommendation I highly recommend moving to the individual files and excecuting them individually on the most interesting data files you have. in the "__main__" part of the code the reduction gets called and then a figure is created that shows you the old data points in blue and the ones that were taken for the reduced data set in red. This helps tweaking the parameters and finding a reduction that is just right for your task.

Now that you have your hyperparameters it is time for the final step to arrive at our final data set. Here you either use create_dataset_peaks.py or create_dataset.py depending whether you chose a _peaks method or a reduce_data_accurate method (such as redaac_every_xth.py).
The script then calls the appropriate function.

Personally I had the best results doing a reduction using reduce_data_accurate.py with positions as metric and choosing the threshold in such a way that the data is reduced to 10%.

In [ ]:
#Excecute for peak samples
from reduce_data.create_dataset_peaks import concat_dataframes
#import here the function you want to use or uncomment and change the 4th parameter to the function
#from reduce_data.reduce_data_peaks_weighted import reduce_redundancy_weight
#from reduce_data.reduce_data_peaks import  reduce_redundancy
from reduce_data.sample_more_peaks import reduce_redundancy_min_max

# Replace with the actual root folder path that contains all pandas dataframes
folder_path = "/home/flo/pacemaker/data_grouped"

concat_dataframes(folder_path, "forces", 0.3, reduce_redundancy_min_max, norm="fro")
print("done")

In [ ]:
#Execute for difference threshold reduction
from reduce_data.create_dataset import concat_dataframes
#from reduce_data.reduce_data_accurate import reduce_redundancy
from reduce_data.redaac_every_xth import  reduce_redundancy

folder_path = "/home/flo/pacemaker/data_grouped/"  # Replace with the actual root folder path
concat_dataframes(folder_path, "forces", 2.5, reduce_redundancy,norm="fro")
print("done")

Now moving to pacemaker we have to create an input file. Follow the instructions here https://pacemaker.readthedocs.io/en/latest/pacemaker/quickstart/#automatic_input_file_generation

In general a cutoff radius of 7A is probably sufficient. I'd recommend to take 5% to 15% for the test data (which is actually more of an validation data if you ask me). As for the weighting scheme you might want to play with either uniform or energy. Personally I did not find much of a difference.

Now we can still change some parameters in the input.yaml file. In general the standard parameters work quite well. I might use a kappa value of 0.6 and if you want to prioritize forces a value of 0.8-0.99 might be even better. Combining this with another run using a lower kappa such as 0.1  is also a good option, since lower kappa prioritizes energys in the weighted loss function.

Also for batch sizes I'd take at least 100 anything below will ruin performance and convergence. On the VSC5 the maximum structures that fit into memory were around 500. PACEMAKER however is able to reduce the batch size to fit into memory, so feel free to aim higher. For optimizers I tried BFGS and Adam, both work well.

At last anywhere from 200 to 600 iterations will give you converged results. Again I'd recommend aim for the higher end. Depending on your data set this will tak 20 hours to finish on an NVIDIA A100 GPU (as in the VSC5).
PACEMAKER has many more parameters, see https://pacemaker.readthedocs.io/en/latest/pacemaker/inputfile/

Personally in my case fit cycles often times broke and ladder schemes also did not really improve my final potential.
Being close to standard parameters seemed to provide the best results.


Submitting the jobs

Results

Testing the results and running simulations

plotting simulations